## Install Packages

In [56]:
!pip install -qU langchain langchain-openai langchain-community beautifulsoup4 faiss-cpu selenium selenium-wire undetected-chromedriver blinker==1.4

## Library

In [57]:
import os
import re
import math
import time
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse as parse_date
from langchain.docstore.document import Document
from google.colab import userdata
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

## Config. Environment

In [58]:
import os
import requests
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from google.colab import userdata

try:
    os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
    os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
    os.environ["AZURE_OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')
    os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
    os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
    print("Azure credentials loaded successfully from Colab Secrets.")
except Exception as e:
    print(f"Could not load secrets. Please ensure you have added all required keys to the Colab Secrets manager. Error: {e}")

Azure credentials loaded successfully from Colab Secrets.


## Data Scraping

In [59]:
# URLs for the release notes
URLS = {
    "simplidots": "https://fitur-sap.simplidots.id/",
    "langflow": "https://api.github.com/repos/langflow-ai/langflow/releases",
    "anthropic": "https://docs.anthropic.com/en/release-notes/api",
    "chatgpt": "https://help.openai.com/en/articles/6825453-chatgpt-release-notes"
}

In [60]:
!pip install -q selenium undetected-chromedriver
!sudo apt-get update -y
!sudo apt-get install -y chromium-chromedriver
!sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

### Headless Chrome Setup

In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
from typing import List

# Langchain-like Document stub
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

### Web Scraper

### GitHub Scraper

In [62]:
def scrape_github_releases(api_url):
    """
    Scrapes GitHub releases and returns a list of Document objects,
    each with its content and release date in the metadata.
    """
    documents = []
    try:
        response = requests.get(f"{api_url}?per_page=15", timeout=15)
        response.raise_for_status()
        releases = response.json()
        for release in releases:
            content = f"## {release.get('name', 'Untitled Release')}\n\n{release.get('body', 'No description.')}"

            # Get the release date directly from the API response
            release_date = release.get('published_at', '')

            # Create a Document for each release
            doc = Document(
                page_content=content,
                metadata={
                    "source": "https://github.com/langflow-ai/langflow/releases",
                    "release_date": release_date.split('T')[0] if release_date else 'unknown' # Format as YYYY-MM-DD
                }
            )
            documents.append(doc)
        return documents
    except requests.RequestException as e:
        print(f"Error fetching GitHub releases from {api_url}: {e}")
        return []

### SimpliDots Selenium + Link Crawler Scraper

In [63]:
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

def scrape_simplidots_with_selenium(base_url, max_depth=2):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    visited = set()
    result_texts = []

    def crawl(url, depth):
        if depth > max_depth or url in visited:
            return
        print(f"Crawling (depth {depth}): {url}")
        visited.add(url)
        try:
            driver.get(url)
            wait_selector = "main.page-has-toc"
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, wait_selector))
            )

            # Use the same selector to find the element
            content = driver.find_element(By.CSS_SELECTOR, wait_selector).text.strip()
            if content:
                result_texts.append(f"URL: {url}\n\n{content}")

            links_to_visit = []
            anchors = driver.find_elements(By.TAG_NAME, "a")
            for a in anchors:
                try:
                    href = a.get_attribute("href")
                    if href and href.startswith(base_url) and href not in visited:
                        links_to_visit.append(href)
                except StaleElementReferenceException:
                    continue

            for link in links_to_visit:
                crawl(link, depth + 1)

        except TimeoutException:
            print(f"Timeout waiting for content on {url}")
            driver.save_screenshot(f"timeout_on_{url.replace('/', '_')}.png")
        except Exception as e:
            print(f"Failed to crawl {url}: {e}")

    try:
        crawl(base_url, 1) # Start crawling from depth 1
    finally:
        driver.quit()

    return "\n\n---\n\n".join(result_texts)

### Anthropic Selenium Scraper

In [64]:
import time

def scrape_anthropic_with_selenium(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(15)  # wait for full load

        body = driver.find_element(By.TAG_NAME, "body")
        return body.text
    except Exception as e:
        print(f"Error using Selenium for Anthropic release notes: {e}")
        return None
    finally:
        driver.quit()

### Execute Scraping

In [65]:
# ### Execute Scraping Cell - UPDATED ###

print("Starting data scraping...")
all_documents = []

# SimpliDOTS via Selenium
simplidots_text = scrape_simplidots_with_selenium(URLS["simplidots"], max_depth=2)
if simplidots_text:
    # We will process the SimpliDOTS text blob later
    all_documents.append(Document(page_content=simplidots_text, metadata={"source": URLS["simplidots"]}))
    print(f"Scraped SimpliDOTS: {len(simplidots_text)} characters")
else:
    print("Failed to scrape SimpliDOTS.")

# Langflow - now returns a list of documents
langflow_docs = scrape_github_releases(URLS["langflow"])
if langflow_docs:
    all_documents.extend(langflow_docs) # Use extend to add all items from the list
    print(f"Scraped Langflow: {len(langflow_docs)} release documents")
else:
    print("Failed to scrape Langflow.")

# Anthropic via Selenium
anthropic_text = scrape_anthropic_with_selenium(URLS["anthropic"])
if anthropic_text:
    all_documents.append(Document(page_content=anthropic_text, metadata={"source": URLS["anthropic"]}))
    print(f"Scraped Anthropic: {len(anthropic_text)} characters")
else:
    print("Failed to scrape Anthropic.")

Starting data scraping...
Crawling (depth 1): https://fitur-sap.simplidots.id/
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/live-mode-kini-dilengkapi-opsi-reset-atau-tidak-reset-data-31-juli-2025
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/beta-integrasi-sales-invoice-simplidots-x-accurate-online-17-juli-2025
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/perbaikan-pemilihan-gudang-pada-buat-sales-invoice-11-juli-2025
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/penambahan-fitur-collection-03-july-2025
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/new-feature-tanya-ai
Crawling (depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh

In [66]:
if simplidots_text:
    words = simplidots_text.split()
    display(" ".join(words[:1000]))
else:
    display("simplidots text was not scraped successfully.")

'URL: https://fitur-sap.simplidots.id/ Copy SMH Fitur pada SMH (Sales Management Hub) Temukan penjelasan mengenai fitur-fitur terbaru di 2024 2023 2025 Next 2025 Last updated 2 months ago Was this helpful? --- URL: https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025 Copy SMH FITUR PADA SMH (SALES MANAGEMENT HUB) 2025 🔜 Live Mode Kini Dilengkapi Opsi Reset atau Tidak Reset Data - 31 Juli 2025 🔥 [Beta] - Integrasi Sales Invoice SimpliDOTS x Accurate Online - [17 Juli 2025] 🔥 Perbaikan Pemilihan Gudang pada Buat Sales Invoice - [11 Juli 2025] 🔥 Penambahan Fitur Collection - [03 July 2025] New Feature: Tanya AI 🚀 Penambahan Fitur Customer Limit - [19 May 2025] 🚀 Penambahan Fitur Log Activity, Pengaturan Quantity, Jam Mulai dan Akhir Promo - [30 April 2025] 🚀 Penambahan Fitur Edit Warehouse & Bulk Transfer Stock - [25 March 2025] 🚀 Penambahan Fitur Impor dan Ekspor Stok - [20 March 2025] 🚀 Custom Mapping Delivery Summary (DS) - [03 March 2025] 🚀 Open API Stock dan U

In [67]:
if langflow_text:
    words = langflow_text.split()
    display(" ".join(words[:1000]))
else:
    display("Langflow text was not scraped successfully.")

"## 1.5.0.post1 <!-- Release notes generated using configuration in .github/release.yml at 1.5.0.post1 --> ## What's Changed ### ✨ New Features * feat: Add dynamic theming support to WatsonxAI icon by @Cristhianzl in https://github.com/langflow-ai/langflow/pull/8935 * feat: jigsawstack bundle integration by @Khurdhula-Harshavardhan in https://github.com/langflow-ai/langflow/pull/8832 * feat: enhance DataFrame Operations component with contains filter and modern UI by @rodrigosnader in https://github.com/langflow-ai/langflow/pull/8838 * feat: add DataFrame output to Structured Output component by @rodrigosnader in https://github.com/langflow-ai/langflow/pull/8842 ### 🐛 Bug Fixes * fix: Improve modal layout responsiveness and overflow handling by @Cristhianzl in https://github.com/langflow-ai/langflow/pull/8936 * fix: Improve flow export error handling and validation by @Cristhianzl in https://github.com/langflow-ai/langflow/pull/8943 * fix: make deletion of single file commit to DB, cre

In [68]:
if anthropic_text:
    words = anthropic_text.split()
    display(" ".join(words[:1000]))
else:
    display("Anthropic text was not scraped successfully.")

'Anthropic home page English Search... Navigation Release Notes API RELEASE NOTES API Copy page Follow along with updates across Anthropic’s API and Developer Console. July 28, 2025 We’ve released text_editor_20250728, an updated text editor tool that fixes some issues from the previous versions and adds an optional max_characters parameter that allows you to control the truncation length when viewing large files. July 24, 2025 We’ve increased rate limits for Claude Opus 4 on the Anthropic API to give you more capacity to build and scale with Claude. For customers with usage tier 1-4 rate limits, these changes apply immediately to your account - no action needed. July 21, 2025 We’ve retired the Claude 2.0, Claude 2.1, and Claude Sonnet 3 models. All requests to these models will now return an error. Read more in our documentation. July 17, 2025 We’ve increased rate limits for Claude Sonnet 4 on the Anthropic API to give you more capacity to build and scale with Claude. For customers wi

## Data Ingestion

### Chunking Process

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
)

print("Chunking documents...")
chunked_docs = text_splitter.split_documents(all_documents)
print(f"Documents chunked successfully. Total chunks: {len(chunked_docs)}")

Chunking documents...
Documents chunked successfully. Total chunks: 602


## Data Preprocessing

### Define Text Cleaning Function

In [70]:
def clean_text(text):
    """Cleans a single string of text."""
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = '\n'.join(line.strip() for line in text.split('\n'))
    artifacts = [
        "Was this helpful?", "Powered by GitBook", "Copy", "Next", "Previous",
        "Last updated", "ago", "hours", "minutes"
    ]
    for artifact in artifacts:
        text = text.replace(artifact, "")
    # Remove any line that is just a year (e.g., '2025', '2024')
    text = re.sub(r'^\d{4}$', '', text, flags=re.MULTILINE)
    return text.strip()

### Format Release Date

In [71]:
def extract_and_format_date(text):
    """
    Finds a date, translates Indonesian months, and returns a formatted string.
    """
    # Mapping for Indonesian to English months
    month_map = {
        'januari': 'january', 'februari': 'february', 'maret': 'march', 'april': 'april',
        'mei': 'may', 'juni': 'june', 'juli': 'july', 'agustus': 'august',
        'september': 'september', 'oktober': 'october', 'november': 'november', 'desember': 'december'
    }

    # Regex to find dates with either English or Indonesian month names
    date_pattern = r"(?i)(\d{1,2}\s+(?:Jan(?:uari)?|Feb(?:ruari)?|Mar(?:et)?|Apr(?:il)?|Mei|Jun(?:i)?|Jul(?:i)?|Agu(?:stus)?|Sep(?:tember)?|Okt(?:ober)?|Nov(?:ember)?|Des(?:ember)?)\s+\d{4}|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{1,2}(?:st|nd|rd|th)?(?:,)?\s+\d{4})"

    match = re.search(date_pattern, text)
    if match:
        try:
            date_str = match.group(0).lower()
            # Translate month if it's Indonesian
            for indo, eng in month_map.items():
                date_str = date_str.replace(indo, eng)

            # Parse the cleaned date string and format it
            parsed_date = parse_date(date_str)
            return parsed_date
        except (ValueError, TypeError):
            return None
    return None

### Initiate Preprocessing

In [72]:
print("Starting data preprocessing and cleaning...")

# First, chunk the documents that haven't been processed yet (SimpliDots, Anthropic, ChatGPT)
docs_to_chunk = [doc for doc in all_documents if 'release_date' not in doc.metadata]
chunked_non_github_docs = text_splitter.split_documents(docs_to_chunk)

# Start our final processed list with the already-good GitHub docs
processed_docs = [doc for doc in all_documents if 'release_date' in doc.metadata]

# Now, process the newly chunked documents
current_date_for_section = None
last_source = None

for doc in chunked_non_github_docs:
    # Reset date when we switch to a new source file
    if doc.metadata['source'] != last_source:
        current_date_for_section = None
        last_source = doc.metadata['source']

    cleaned_content = clean_text(doc.page_content)
    if len(cleaned_content) < 50:
        continue

    extracted_date = extract_and_format_date(cleaned_content)
    if extracted_date:
        current_date_for_section = extracted_date

    if current_date_for_section:
        doc.metadata['release_date'] = current_date_for_section.strftime('%Y-%m-%d')
    else:
        doc.metadata['release_date'] = 'unknown'

    doc.page_content = cleaned_content
    processed_docs.append(doc)

print(f"Preprocessing complete. Total processed documents/chunks: {len(processed_docs)}")

# Sort all documents by date, newest first
processed_docs.sort(key=lambda x: x.metadata.get('release_date', '0000-00-00'), reverse=True)

print("\nExample of a newly processed chunk's metadata:")
# Find the first non-GitHub doc to show a successful example
for doc in processed_docs:
    if "github" not in doc.metadata["source"]:
        print(doc.metadata)
        break

Starting data preprocessing and cleaning...
Preprocessing complete. Total processed documents/chunks: 400

Example of a newly processed chunk's metadata:
{'source': 'https://fitur-sap.simplidots.id/', 'release_date': '2025-07-31'}


### Data Embedding

In [73]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import math
import time

print("Initializing Azure OpenAI Embeddings model...")
azure_embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)
print("Embedding model initialized.")

# Batch processing and embedding
batch_size = 1000
total_chunks = len(chunked_docs)
vector_store = None # Initialize vector_store to None

if total_chunks > 0:
    num_batches = math.ceil(total_chunks / batch_size)
    print(f"\nStarting embedding process in {num_batches} batches of size {batch_size}...")

    for i in range(0, total_chunks, batch_size):
        batch_number = (i // batch_size) + 1
        start_time = time.time()

        # Get the current batch of documents
        batch_docs = chunked_docs[i:i + batch_size]
        print(f"  - Processing Batch {batch_number}/{num_batches} ({len(batch_docs)} chunks)...")

        if vector_store is None:
            # For the first batch, create the FAISS index
            vector_store = FAISS.from_documents(batch_docs, azure_embeddings)
            print("    - Initial FAISS index created.")
        else:
            # For subsequent batches, add them to the existing index
            vector_store.add_documents(batch_docs)
            print("    - Batch added to existing FAISS index.")

        end_time = time.time()
        print(f"  - Batch {batch_number} finished in {end_time - start_time:.2f} seconds.")

    print("\nAll batches have been processed and embedded.")

    # Save the completed vector store
    vector_store.save_local("faiss_index_release_notes")
    print("Vector store saved to Colab's local directory: 'faiss_index_release_notes'")
else:
    print("No documents were chunked. Skipping embedding process.")

Initializing Azure OpenAI Embeddings model...
Embedding model initialized.

Starting embedding process in 1 batches of size 1000...
  - Processing Batch 1/1 (602 chunks)...
    - Initial FAISS index created.
  - Batch 1 finished in 6.97 seconds.

All batches have been processed and embedded.
Vector store saved to Colab's local directory: 'faiss_index_release_notes'


## Initialize RAG System

### Initialize LLM

In [74]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Explicitly get and set the variables
azure_endpoint = userdata.get('AZURE_OPENAI_ENDPOINT')
azure_deployment = userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
api_key = userdata.get('AZURE_OPENAI_API_KEY')
api_version = userdata.get('OPENAI_API_VERSION')

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    api_key=api_key,
    api_version=api_version,
    model=f"azure/{userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')}"
)

print("LLM initialized.")

# Create a retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={'k': 4})
print("Retriever created.")

LLM initialized.
Retriever created.


### RAG Chain

In [81]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rewrite_template = """
You are an expert query translator. Your task is to translate the user's question into a clean, direct English search query.
Identify the key software names mentioned (like SimpliDots, LangFlow, etc.) and the user's core intent (e.g., "latest updates", "new features").
Do NOT add any new technical concepts or topics that are not in the original question. Keep the query focused on the original's intent.

Original Question:
{question}

Cleaned English Search Query:
"""

rewrite_prompt = PromptTemplate(
    template=rewrite_template,
    input_variables=["question"],
)

# The rewriter chain itself
query_rewriter = rewrite_prompt | llm | StrOutputParser()
print("Constrained query rewriter chain created.")


# --- 2. Define the Main RAG Prompt ---
# This prompt remains the same.
prompt_template = """
You are an intelligent assistant for querying software release notes.
Use only the following retrieved context to answer the user's question. If the question is in Indonesian, please answer in Indonesian.
If you don't have enough information from the context for a specific topic, state that clearly for that topic and answer the rest.
Do not make up information. Be concise and helpful.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Helper function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(f"Source: {doc.metadata.get('source', 'N/A')}\nDate: {doc.metadata.get('release_date', 'N/A')}\nContent: {doc.page_content}" for doc in docs)

rag_chain = (
    {
        "context": query_rewriter | retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("Final RAG chain with constrained rewriting created. Ready to answer questions.")

Constrained query rewriter chain created.
Final RAG chain with constrained rewriting created. Ready to answer questions.


## LLM Testing

In [86]:
if 'rag_chain' in locals():
    # --- Get user input for the query ---
    query = input("Enter your query: ")
    print(f"Question: {query}")

    # You can also see what the rewriter does with your query:
    rewritten_query = query_rewriter.invoke(query)
    print(f"Rewritten Query for Retrieval: {rewritten_query}\n")

    # Invoke the full chain
    answer = rag_chain.invoke(query)

    print("\nAnswer:")
    print(answer)
else:
    print("Cannot run tests because the RAG chain was not created.")

Enter your query: Hello, apa aja ya update SimpliDots ditahun 2025?
Question: Hello, apa aja ya update SimpliDots ditahun 2025?
Rewritten Query for Retrieval: SimpliDots 2025 updates


Answer:
Pada tahun 2025, update dari SimpliDOTS yang tersedia dalam konteks adalah:

1. **[Beta] - Integrasi Sales Invoice SimpliDOTS x Accurate Online (17 Juli 2025)**  
   - SimpliDOTS sedang mengembangkan integrasi dengan Accurate Online untuk fitur Sales Invoice (Faktur Penjualan).  
   - Tujuan fitur ini adalah mempermudah proses penagihan tanpa perlu input manual di dua tempat.  
   - Fitur ini masih dalam tahap BETA, dan koneksi ke sistem pembayaran (Customer Payment) belum tersedia.  
   - Pengguna dapat mulai mencoba fitur ini dan memberikan masukan.

Tidak ada informasi tambahan untuk update lainnya di tahun 2025 dalam konteks ini.
